# TrainScript GoNet

In [24]:
import numpy as np
import IPython.display
import sqlite3
import io

import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F

In [25]:
torch.cuda.is_available()

True

In [33]:
# load data
def convert_array(text):
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)

# Converts TEXT to np.array when selecting
sqlite3.register_converter("array", convert_array)

con = sqlite3.connect(r"DB/Move/dan_data_295", detect_types=sqlite3.PARSE_DECLTYPES)
cur = con.cursor()
cur.execute("SELECT * FROM movedata")
data = cur.fetchall()
con.close()

# get rid of index of db_entry and create N*892-dim input array
for ind in range(len(data)):
    data[ind] = np.concatenate(data[ind][1:])

data = np.array(data)

In [131]:
d2[:,:81].reshape((d2.shape[0],9,9)).shape

(15704, 9, 9)

In [208]:
# load all data, shape is 81 input 82 output 162: filtered boards
numberOfSamples = 1000
labels = torch.LongTensor(loadedData[:numberOfSamples,81:163])
data = torch.FloatTensor(loadedData[:numberOfSamples,:81].reshape((numberOfSamples,9,9)))
data = data.unsqueeze(1) # add fake batch dimension
data.shape, labels.shape 

(torch.Size([1000, 1, 9, 9]), torch.Size([1000, 82]))

In [209]:
import torch.utils.data as data_utils

train = data_utils.TensorDataset(data, labels)
train_loader = data_utils.DataLoader(train, batch_size=16, shuffle=True)

In [210]:
# the Classifier stolen from Pytorch Tutorial

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3 square convolution
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 12, 3)
        
        # output channels 36 * (5 * 5) -> output per channel  
        self.fc1 = nn.Linear(12 * 5 * 5, 82)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        # Flatten 2d array to pass to linear layer, -1 works for arbitrary #rows which can vary with batchsize
        x = x.view(-1, self.num_flat_features(x))
        x = self.fc1(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=300, out_features=82, bias=True)
)


In [211]:
# training
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [212]:
for epoch in range(100):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, torch.max(labels,1)[1])
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

Finished Training


In [216]:
net(torch.FloatTensor(np.zeros((9,9))).unsqueeze(0).unsqueeze(0))

tensor([[-1.1568, -0.5298, -0.4719, -0.7667, -1.5456, -0.5871, -0.4048, -0.5829,
         -1.0691, -0.6141, -0.0178,  0.2488,  0.5656, -0.2305,  0.5626,  0.2647,
         -0.0071, -0.4808, -0.5404,  0.1986,  0.8091,  0.6497,  0.5500,  0.6345,
          0.8404,  0.2590, -0.4320, -0.6044,  0.5514,  0.6463,  0.6456,  0.6316,
          0.6528,  0.6508,  0.5466, -0.7506, -1.5295, -0.2022,  0.5426,  0.6240,
          0.3698,  0.6360,  0.5557, -0.2558, -1.4251, -0.5772,  0.5691,  0.6594,
          0.6433,  0.6303,  0.6183,  0.6626,  0.5764, -0.6674, -0.4320,  0.2799,
          0.8504,  0.6342,  0.5417,  0.6337,  0.8542,  0.2112, -0.4178, -0.6264,
         -0.0202,  0.2144,  0.5566, -0.2272,  0.5484,  0.2662, -0.0302, -0.6856,
         -1.1484, -0.5921, -0.3418, -0.6853, -1.5480, -0.6850, -0.4884, -0.5753,
         -1.1595,  1.3502]], grad_fn=<ThAddmmBackward>)